In [1]:
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import players
import pandas as pd
import time

# Research Question

How Consistent do NBA Players Perform across different games?

In [2]:
def get_player_id(name):
    player_dict = players.find_players_by_full_name(name)
    return player_dict[0]['id'] if player_dict else None

def get_player_game_logs(player_name, season='2023-24'):
    player_id = get_player_id(player_name)
    if not player_id:
        print(f"No player found with name {player_name}")
        return None

    logs = playergamelog.PlayerGameLog(player_id=player_id, season=season, season_type_all_star='Regular Season')
    df = logs.get_data_frames()[0]
    return df

In [3]:
player_name = 'Stephen Curry'  # You can change this
game_logs = get_player_game_logs(player_name)

In [4]:
game_logs['GAME_DATE'] = pd.to_datetime(game_logs['GAME_DATE'])
game_logs['HOME'] = game_logs['MATCHUP'].apply(lambda x: 'vs.' in x)
game_logs['OPPONENT'] = game_logs['MATCHUP'].apply(lambda x: x.split()[-1])

C:\Users\ericz\AppData\Local\Temp\ipykernel_6956\2018404500.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  game_logs['GAME_DATE'] = pd.to_datetime(game_logs['GAME_DATE'])


In [5]:
game_logs[['GAME_DATE', 'MATCHUP', 'HOME', 'OPPONENT', 'PTS', 'REB', 'AST', 'FG_PCT', 'PLUS_MINUS']].head()

,GAME_DATE,MATCHUP,HOME,OPPONENT,PTS,REB,AST,FG_PCT,PLUS_MINUS
0,2024-04-12,GSW vs. NOP,True,NOP,33,4,5,0.522,-8
1,2024-04-11,GSW @ POR,False,POR,22,7,8,0.364,7
2,2024-04-09,GSW @ LAL,False,LAL,23,7,8,0.778,13
3,2024-04-05,GSW @ DAL,False,DAL,28,6,5,0.391,-5
4,2024-04-04,GSW @ HOU,False,HOU,29,6,6,0.643,29


In [6]:
home_away = game_logs.groupby('HOME')[['PTS', 'FG_PCT', 'REB', 'AST']].mean()
print("Home vs Away Averages:")
print(home_away)

Home vs Away Averages:
             PTS    FG_PCT       REB       AST
HOME                                          
False  25.657895  0.441816  4.605263  5.578947
True   27.250000  0.447111  4.305556  4.638889


In [7]:
consistency = game_logs[['PTS', 'REB', 'AST']].std()
print("Statistical Variability (Standard Deviation):")
print(consistency)

Statistical Variability (Standard Deviation):
PTS    9.583206
REB    2.382523
AST    2.421098
dtype: float64


In [8]:
matchup_perf = game_logs.groupby('OPPONENT')[['PTS', 'FG_PCT']].mean().sort_values('PTS', ascending=False)
matchup_perf.head(10)

,PTS,FG_PCT
OPPONENT,,
ATL,42.500000,0.508500
MIN,34.500000,0.434500
SAS,34.000000,0.536000
IND,33.500000,0.528500
LAL,33.000000,0.566000
BKN,33.000000,0.568000
SAC,31.000000,0.526000
NOP,30.000000,0.504000
DET,30.000000,0.415000
